In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Taskmaster
#!pip install tensorflow --upgrade
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [1]:
import rubiks as rubiks
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from rubiks_network_experimental import student_network
from utils import dotdict
from keras import regularizers

task = rubiks.rubiks_task
setup = rubiks.rubiks_setup
rng = np.random.default_rng(seed=0)

core_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 512,
            "learning_rate" : 0.001,
            "residual_layers" : 2
})

value_network_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 512,
            "learning_rate" : 0.001,
            "post_core_residual_layers" : 2,
            "reward_fork_layers": 1,
            "value_fork_layers": 1
})

state_network_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 512,
            "learning_rate" : 0.001,
            "post_core_residual_layers" : 2
})

params = dotdict({
    "core_params":core_params,
    "value_network_params": value_network_params,
    "state_network_params": state_network_params,
    "state_size":task.input_size,
    "action_codes": task.n_actions
})

my_student_network = student_network.create(params)
#my_student_network = student_network.load("models/combined/trained/trained")
student_template = student(task, 50,my_student_network,0.95)
t = teacher(setup, lambda n : 1+rng.poisson(lam=5,size=n))
c = classroom(task, setup, t, student_template, n_students=1, max_steps=10, buffer_size = lambda n : np.minimum(4096,np.maximum(1024,n//2)))

ValueError: Inputs have incompatible shapes. Received shapes (16, 16, 32) and (16, 16, 64)

In [6]:
c.max_steps=10
c.buffer_size = lambda n : np.minimum(4096,np.maximum(1024,n//2))
t.step_dist = lambda n : 5*np.ones(n,dtype=int)

In [7]:

for i in range(50):
    print(f"Round {i}.")
    c.run_training_batch(n_problems=100,epochs_per_episode=3)
    #if i % 10 == 9:
    #    my_student_network.save("models/trained.h5")

Round 0.
Before step 1, 100 out of 100 remain open.
Before step 2, 99 out of 100 remain open.
Before step 3, 95 out of 100 remain open.
Before step 4, 86 out of 100 remain open.
Before step 5, 72 out of 100 remain open.
Before step 6, 65 out of 100 remain open.
Before step 7, 56 out of 100 remain open.
Before step 8, 52 out of 100 remain open.
Before step 9, 50 out of 100 remain open.
Before step 10, 50 out of 100 remain open.
After step 10, 50 out of 100 remain open.
Epoch 1/3
55/55 [==============================] - 2s 30ms/step - loss: 0.0878 - value_loss: 0.0514 - reward_loss: 0.0082 - confidence_loss: 0.0282
Epoch 2/3
55/55 [==============================] - 2s 29ms/step - loss: 0.0522 - value_loss: 0.0201 - reward_loss: 0.0074 - confidence_loss: 0.0247
Epoch 3/3
55/55 [==============================] - 2s 30ms/step - loss: 0.0448 - value_loss: 0.0143 - reward_loss: 0.0063 - confidence_loss: 0.0242
Round 1.
Before step 1, 100 out of 100 remain open.
Before step 2, 100 out of 100 r

KeyboardInterrupt: 

In [4]:
n=8192
rng = np.random.default_rng()
states = np.arange(task.input_size)[None,:].repeat(n,axis=0)

for k in range(20):
    print(f"Round {k}.")
    #c.run_training_batch(n_problems=100,epochs_per_episode=1)

    for i in range(50):
        _, states = rubiks.task_action(states,rng.choice(a=task.n_actions,size=n))

    actions = rng.choice(a=task.n_actions,size=n)
    _, next_states = rubiks.task_action(states,actions)

    actions = (np.arange(task.n_actions) == actions[...,None]).astype(float)
    colors = rubiks.make_neural_input(states)
    next_colors = rubiks.make_neural_input(next_states)
    my_student_network.fit_state(colors,actions,next_colors,epochs = 2)

    #if k % 10 == 9:
    #  my_student_network.save("models/combined/trained")

Round 0.
Epoch 1/2
120/120 [==============================] - 7s 61ms/step - loss: 0.1458 - val_loss: 0.1500
Epoch 2/2
120/120 [==============================] - 7s 59ms/step - loss: 0.1229 - val_loss: 0.1263
Round 1.
Epoch 1/2
120/120 [==============================] - 7s 59ms/step - loss: 0.1306 - val_loss: 0.1371
Epoch 2/2
120/120 [==============================] - 7s 59ms/step - loss: 0.1142 - val_loss: 0.1178
Round 2.
Epoch 1/2
120/120 [==============================] - 7s 61ms/step - loss: 0.1274 - val_loss: 0.1371
Epoch 2/2
120/120 [==============================] - 7s 59ms/step - loss: 0.1181 - val_loss: 0.1238
Round 3.
Epoch 1/2
120/120 [==============================] - 7s 60ms/step - loss: 0.1327 - val_loss: 0.1418
Epoch 2/2
120/120 [==============================] - 7s 60ms/step - loss: 0.1211 - val_loss: 0.1268
Round 4.
Epoch 1/2
120/120 [==============================] - 7s 60ms/step - loss: 0.1310 - val_loss: 0.1363
Epoch 2/2
120/120 [==============================] - 7s

KeyboardInterrupt: 

In [5]:
from rubiks import print_coloring
states =  np.arange(54,dtype=int)[None,:]
actions = np.array([9],dtype=int)
prediction = my_student_network.predict_state(task.make_input(states),(np.arange(task.n_actions) == actions[...,None]).astype(float))
print_coloring(prediction.reshape(54,6).argmax(axis=-1),color_array=True)
print_coloring(rubiks.task_action(states,actions)[1][0])

1/1 [==============================] - 0s 349ms/step
   BRB      
   BBB      
   BBB      
RRRWWWOOOYYY
WWWOOOYYYRRR
RRRWWWOOOYYY
   GGG      
   GGG      
   GGG      

   BBB      
   BBB      
   BBB      
RRRWWWOOOYYY
WWWOOOYYYRRR
RRRWWWOOOYYY
   GGG      
   GGG      
   GGG      



In [15]:
prediction.reshape(54,6)[11]

array([1.0267684e-05, 9.0565451e-04, 1.5079054e-04, 4.5128591e-02,
       9.5010392e-02, 1.2742223e-01], dtype=float32)

In [6]:
from rubiks import print_coloring
states =  np.arange(54,dtype=int)[None,:]
actions = np.array([9],dtype=int)
my_student_network.predict_state(task.make_input(states),(np.arange(task.n_actions) == actions[...,None]).astype(float)).reshape(54,6)

1/1 [==============================] - 0s 40ms/step


array([[1.00000000e+00, 2.70621442e-10, 4.97854633e-13, 2.06695161e-10,
        2.17686494e-11, 3.36848494e-10],
       [1.49457524e-09, 8.51330006e-10, 9.99998808e-01, 7.95608557e-08,
        1.70973902e-09, 1.06619984e-06],
       [2.32725855e-04, 2.70772193e-06, 8.17318723e-05, 2.33141836e-05,
        9.99641299e-01, 1.82437834e-05],
       [1.00000000e+00, 4.03902618e-08, 1.98201034e-13, 1.63797029e-15,
        5.76395182e-11, 7.81352316e-09],
       [1.02120352e-13, 1.54375568e-10, 1.00000000e+00, 1.94999347e-11,
        8.15008044e-12, 9.77136150e-10],
       [9.99999881e-01, 7.54530305e-08, 1.53104429e-09, 1.17643486e-08,
        3.18252473e-11, 2.02680384e-09],
       [3.10112767e-07, 2.61199119e-07, 9.99999166e-01, 1.60647062e-07,
        2.19735050e-10, 3.99436608e-08],
       [3.99852870e-05, 9.59444506e-08, 1.47011392e-07, 6.02884086e-07,
        1.21217806e-06, 9.99957919e-01],
       [1.86581208e-04, 6.43006861e-05, 2.53230886e-04, 1.62349941e-04,
        9.81153250e-01, 